In [1]:
import numpy as np
import rasterio
from rasterio.mask import mask


from rasterio.features import bounds as feature_bounds
import pandas as pd

import geopandas as gpd
import fiona

import numpy.ma as ma # for masked arrays. This allows us to handle NAs for integer arrays


# for the blur
from scipy.ndimage import convolve, median_filter

#todo: align the subnational subtiffs to the overall 100m tif. 

In [2]:
# # input data:
# probability maps (as a list of paths)
# fixed pixels maps
# admin polygon or rasterized --> they need to have columns cl_1 to cl_n containing the number of pixels (or area?)
# admin totals

# 2 lists of integers giving the probability layers and the fixed layers the class intergers in order!

# # input parameters:
# type of blur / kernel



# output: final map

In [3]:
# lets write the function for a single element first. 

# check for raster alignment first!!? how. 
# check if polys number of cl_columns mathces the number of layers

# somehow check the classes
#   number of fixed pixel and probability layers must equal the number of admin stats. 
#   the fixed ones could be inferred. 

# make sure 255 is the NA value
# make sure the croped  layers have the same number of masked pixels??

# stitching back together the nuts regions


In [286]:
!gdalinfo in_data/Corine_landcover/Grass_ptobability_final_layer_Poland.tif | grep Origin
!gdalinfo in_data/Corine_landcover/Grass_ptobability_final_layer_Poland.tif | grep Size

Origin = (4596000.000000000000000,3556200.000000000000000)
Size is 7163, 6103
Pixel Size = (100.000000000000000,-100.000000000000000)


In [287]:
# check alignment manually
!gdalinfo in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/PL_urbanmask_1xx.tif | grep Origin
!gdalinfo in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/PL_urbanmask_1xx.tif | grep Size

Origin = (4598000.000000000000000,3554000.000000000000000)
Size is 7120, 6060
Pixel Size = (100.000000000000000,-100.000000000000000)


In [207]:
# parameters:
windowsize_median = 3
blurtype = 'median' # unused

In [208]:
admin_GIS_path = 'in_data/admin/PL_stats.geojson'
current_nutsid = 'PL72'

In [184]:
polys = gpd.read_file(admin_GIS_path)

In [194]:
poly = polys.loc[polys['id'] == 'PL72', ]

In [185]:
urban_mask_path = 'in_data/CLC/u2018_clc2018_v2020_20u1_raster100m/DATA/PL_urbanmask_1xx.tif'

In [186]:
foresttif_path    = "out_data/PL_expert_map_integer.tif"
grasslandtif_path = "in_data/Corine_landcover/Grass_ptobability_final_layer_Poland.tif"
croplandtif_path  = "in_data/Corine_landcover/Crops_ptobability_final_layer_Poland.tif"


In [187]:
polys.columns

Index(['id', 'NUTS_ID', 'LEVL_CODE', 'CNTR_CODE', 'NAME_LATN', 'NUTS_NAME',
       'MOUNT_TYPE', 'URBN_TYPE', 'COAST_TYPE', 'FID', 'numeric_id', 'X1990',
       'X1991', 'X1992', 'X1993', 'X1994', 'X1995', 'X1996', 'X1997', 'X1998',
       'X1999', 'X2000', 'X2001', 'X2002', 'X2003', 'X2004', 'X2005', 'X2006',
       'X2007', 'X2008', 'X2009', 'X2010', 'X2011', 'X2012', 'X2013', 'X2014',
       'X2015', 'X2016', 'X2017', 'X2018', 'X2019', 'X2020', 'X2021', 'X2022',
       'cl_1_total', 'cl_2_total', 'cl_3_total', 'cl_4_total', 'geometry'],
      dtype='object')

In [188]:
# 0 = -----
# 1 = forest;
# 2 = arable cropland;
# 3 = permanent cropland;
# 4 = grassland;
# 5 = built-up; 
# 6 = other natural land; 
# 7 = the rest/not releva


In [189]:
fixed_layer_paths = [urban_mask_path]
fixed_layer_codes = [5]

probability_layers_paths = [foresttif_path, grasslandtif_path, croplandtif_path]
probability_layers_codes = [1, 4, 2]


In [196]:
class Input_Data_Exception(Exception):
    pass

def check_admin_data():
    all_classes = fixed_layers_codes + probability_layers_codes

    # check for NAs in the cl columns
    
    # check if all classes are in admin data
    for cl in probability_layers_codes:
        if not f"cl_{cl}_total" in poly.columns:
            print(f"cl_{cl}_total is not in the polys file")
            return False
        
    return True

In [197]:
check_admin_data()

True

In [192]:
def check_input_layers():
    length_probability_layers =  len(probability_layers_paths) == len(probability_layers_codes)
    length_fixed_layers       =  len(fixed_layer_paths) == len(fixed_layers_codes)
    
    all_classes = fixed_layers_codes + probability_layers_codes
    
    duplicates = len(all_classes) == len(set(all_classes))
    
    
    
    return all([length_probability_layers, length_fixed_layers, duplicates])



In [193]:
check_input_layers()

True

In [205]:
type(geometry.iloc[0])

shapely.geometry.multipolygon.MultiPolygon

In [277]:

input_path = probability_layers_paths[0]

def read_and_smear_raster(input_path, poly):
    with rasterio.open(input_path) as src:
            geometry = poly.geometry.iloc[0]
            nutsid = poly['id']

            # Mask and crop the raster using the current polygon
            out_image, out_transform = mask(src, [geometry], invert=False, crop=True)

            out_image = out_image.astype(float)  # Ensure the data is in float
            out_image[out_image == 255] = np.nan


            resolution = src.res  # (pixel width, pixel height)

            # Get CRS
            crs = src.crs
            if crs is not None:
                # Extract EPSG code
                epsg_code = crs.to_epsg()  # This will be None if the EPSG code can't be determined


            affine = out_transform

            if len(out_image.shape) in [1 , 3]:
                raise Input_Data_Exception

            if len(out_image.shape) == 3:
                array = np.squeeze(out_image, axis = 0)

            # mask the 255 values
            array_mask = np.isnan(array)

            array = ma.array(array, mask = array_mask)

            # now blur
            smeared_array  = median_filter(array, size= windowsize_median)

            # Reapply the mask to the blurred band
            smeared_array = ma.array(smeared_array, mask = array_mask)


            return smeared_array


In [264]:
# make a stack of smeared probability layers
probability_layers = np.ma.stack([read_and_smear_raster(layer, poly) for layer in probability_layers_paths])


In [276]:
layer_idx_2_final = {position:classlabel for (position, classlabel) in enumerate(probability_layers_codes)}

rev_layer_idx_2_final = {classlabel:position for (position, classlabel) in enumerate(probability_layers_codes)}


In [279]:
# Flatten the array into a list of (row_index, column_index, value) tuples

def order_probability_layer_pixels(probability_layers):

    # take the probability layer stack and extract the pixel values, together with the x,y,z (layer) location. 
    # also translate the layer postion (i) to the final class code. 
    
    flattened_with_indices = [(layer_idx_2_final[i], j, k, probability_layers[i, j, k]) 
                              for i in range(probability_layers.shape[0]) # layers 
                              for j in range(probability_layers.shape[1]) # rows
                              for k in range(probability_layers.shape[2]) # cols
                              if not ma.is_masked(probability_layers[i, j, k]) and not np.isnan(probability_layers[i, j, k]) ]
    
    # Order the values by the value, which is the 3rd (starting at 0) element of each tuple of the list. 
    flattened_sorted = sorted(flattened_with_indices, key=lambda x: x[3], reverse=True)
       
    
    return flattened_sorted

In [92]:
# Linda asked me to hard-code fix the urban areas. 
# load an urban mask. flatten it, only take the urban pixels and put them at the top of the flattened_sorted list so they get assigned first
# and added to the seen pixels set automatically. 

with rasterio.open(urban_mask_path) as src:
    urban_mask, urban_mask_transform = mask(src, [geometry], invert=False, crop=True)
    
#     urban_mask = ma.array(urban_mask, mask=array_mask[0,:,:])


fixed_pixels =  [(5, j, k, 100* urban_mask[0, j, k]) 
                              for j in range(masked_array.shape[1]) 
                              for k in range(masked_array.shape[2]) 
                              if not ma.is_masked(masked_array[0, j, k]) and urban_mask[0, j, k] == 1  ]


len(fixed_pixels)

84238

In [93]:
array.shape

(1272, 1497)

In [94]:
urban_mask.shape

(1, 1272, 1497)

In [95]:
flattened_sorted = fixed_pixels +  flattened_sorted 

flattened_sorted[:10]

[(5, 10, 415, 100),
 (5, 11, 415, 100),
 (5, 12, 415, 100),
 (5, 12, 423, 100),
 (5, 12, 424, 100),
 (5, 13, 415, 100),
 (5, 13, 416, 100),
 (5, 13, 417, 100),
 (5, 13, 418, 100),
 (5, 13, 419, 100)]

In [ ]:
# 0 = -----
# 1 = forest;
# 2 = arable cropland;
# 3 = permanent cropland;
# 4 = grassland;
# 5 = built-up; 
# 6 = other natural land; 
# 7 = the rest/not releva


In [97]:
flattened_sorted[15524]

(5, 330, 755, 100)

In [103]:
NUTSID = "PL91"
seen_px = set()
output = np.full(array3d.shape[1:3], 10, dtype = 'int')
output = ma.array(output, mask=array_mask[0])


for i, tupl in enumerate(flattened_sorted):
    idx = tupl[1:3]
    
    if idx in seen_px or counter[tupl[0]] < 1:
        continue
    else:
        output[idx] = tupl[0]
        seen_px.add(idx)
        counter[tupl[0]] -= 1

print(counter)

[0, 0, 0, 0, 0, 0, 0, 0]


In [104]:
from rasterio.transform import from_origin

data = output



# Define the transformation and metadata
transform = from_origin(west, north, resolution[0], resolution[1])
height, width = data.shape
crs = f"EPSG:{epsg_code}"  # Example CRS - replace with the appropriate CRS for your data

# Metadata dictionary
metadata = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': 'int16',
    'crs': crs,
    'transform': transform,
    'nodata': 255 
}

# Write to a new TIFF file
with rasterio.open('out_data/predictions/landcover_fillup/PL72_prediction_kernel_newclasses.tif', 'w', **metadata) as dst:
    dst.write(data, 1)


In [60]:
# Counting the frequency of each element
unique_elements, counts = np.unique(output, return_counts=True)

# Creating a dictionary for better readability
frequency = dict(zip(['grassland', 'cropland', 'forest', 'urban', 'misc', 'na'], counts))

print(frequency)